# Transfer Learning using Inception_v3 Pretrained
Transfer learning using Inception_V3 pretrained network weights, by using this methodology you can use any other pretrained heavy weight model to classify your own smaller dataset. There are lots of articles which provide you retrain file like a blackbox but in this i tried to clarify how you actualy can use transfer learning concept in [jupyter notebook](https://github.com/abirjameel/Transfer_Learning) step by step. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
import pandas as pd
import pickle
import os 
import cv2
import math
%matplotlib inline

C:\Users\jamee\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Why Transfer Learning ?
Generally, it is quite difficult to train an entire Convolutional Network from scratch, because it requires large Datasets and enormous amount of computing capacity of GPU’s still it will run for few weeks! These days it is common to use, pretrained ConvNet on a very large dataset e.g. Imagenet which contains 1.2 million images with 1000 categories, as an initialization or a fixed feature extractor usage can be limited to similar set of problems. The three major Transfer Learning techniques are as follows:
ConvNet - feature extractor. Take a pretrained ConvNet on ImageNet, remove the last fully-connected layer (this layer’s outputs are the 1000 classes), then treat the rest of the ConvNet as a fixed feature extractor for the new image dataset. 
Once you have extracted features, the next step is to train a machine learning classifier e.g. SVM, XGboost or Random Forest for new dataset.
Training last layer of ConvNet. The other strategy is to replace and retrain the classifier on top of the pretrained ConvNet on new dataset, as well as fine tuning the weights of the pretrained network by allowing backpropagation. 
You can fine tune all or only selected layers of ConvNet, but in this article you will be training only the last layer. 
Google (has published Inception) and other individual researchers used to publish their trained ConvNet weights for the others to use. 
Andrew Ng, professor at Stanford, said during his widely popular NIPS 2016 tutorial that transfer learning will be the next driver of Machine Learning commercial success.

## How you actually retrain a inception_v3 model on a new dataset  
In this article you are going to retrain inception_v3 model on new datset with new classes, since inception_v3 model is made for some 1000 classes, here you will be using flowers dataset from kaggle datasets https://www.kaggle.com/alxmamaev/flowers-recognition/data.   

The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion. Since Inception_v3 takes 299 x 299 x 3 shape images and these images are of different sizes wo need to do a little bit of preprocessing, you need not to worry about this you can download preprocessed images data from this articles github repository.   

You can start with downloading the pretrained inception model from here http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz untar and save this in a directory, which you can use to load and train this inception model for your new classes.   

## Data Prep

In [ ]:
def create_traindata(num_data = 1000, IMG_SIZE = 299, TRAIN_DIR = ".\\train"):
    training_data = []
    training_label = []
    x = 0
    for directory in os.listdir(TRAIN_DIR):
        path1 = os.path.join(TRAIN_DIR, directory)
        for i, img in enumerate(os.listdir(path1)):
            if i < num_data:
                label = directory
                path2 = os.path.join(path1, img)
                img = cv2.imread(path2)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                img = ((2*img)/255) - 1
                training_label.append(np.array(label))
                training_data.append(np.array(img))
    print(img.shape)
    print("{0} Files loaded".format(num_data))
    return np.array(training_data), np.array(training_label)

In [ ]:
train, label = create_traindata(num_data=1300, IMG_SIZE=299, TRAIN_DIR="flowers/")

In [ ]:
def shuffle_data(X, Y):
    m = X.shape[0]
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,]
    return shuffled_X, shuffled_Y

In [ ]:
train, label = shuffle_data(train, label)

In [ ]:
print(train.shape)
print(label.shape)

In [ ]:
label[0:3]

In [ ]:
label_df = pd.DataFrame(label, columns = ['classes'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
en = LabelEncoder()
en.fit(label_df.classes)
label = en.transform(label_df.classes)

In [ ]:
label[0:10]

In [ ]:
plt.imshow(abs(train[2]))

In [1]:
INCEPTION_V3_CHECKPOINT_PATH = "path_to_inception_directory/inception_v3.ckpt"

it is required to define a few placeholders such as "training" and "X" placeholder that we need to load Inception_v3 graph, X placeholder is used to give inputs to model. 

In [ ]:
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim

X = tf.placeholder(tf.float32, shape=[None, 299, 299, 3], name="X")

training = tf.placeholder_with_default(False, shape=[])
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)
    
inception_saver = tf.train.Saver()

Now we need to find the point in the graph where we should attach the new output layer.  It should be the layer right before the current output layer. One way to do this is to explore the output layer's dimensions.  

In [ ]:
end_points

as you can see this the layer we were searching just before last layer or softmax layer 
`'PreLogits': <tf.Tensor 'InceptionV3/Logits/Dropout_1b/cond/Merge:0' shape=(?, 1, 1, 2048) dtype=float32>` and the name is `PreLogits`. Lets go ahead and do it!

We can drop the 2nd and 3rd dimensions using the `tf.squeeze()` function, Then we can add the final fully connected layer for new dataset classes on top of this layer.

In [ ]:
prelogits = tf.squeeze(end_points["PreLogits"], axis=[1, 2])

In [ ]:
n_outputs = 5 # Since we have 5 classes in flowers dataset 

with tf.name_scope("new_output_layer"):
    flower_logits = tf.layers.dense(prelogits, n_outputs, name="flower_logits")
    Y_proba = tf.nn.softmax(flower_logits, name="Y_proba")

Finally, you need to add the following variables and placeholders to the model:  
- the placeholder for the target variable (Y),
- the loss function, which is the cross-entropy, as usual for a classification task,
- an optimizer, that we use to create a training operation that will minimize the cost function,
- a couple operations to measure the model's accuracy,
- and, finally an initializer and a saver.

Since you want to train only the new output layer you must freeze all layers except the last one (the new last one!), you can do this by passing the list of variables to train to the optimizer `minimize()` method. 

In [ ]:
Y = tf.placeholder(tf.int32, shape=[None,])

with tf.name_scope("train"):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=flower_logits, labels=Y)
    loss = tf.reduce_mean(cross_entropy)
    optimizer = tf.train.AdamOptimizer()
    new_cifar_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="flower_logits")
    training_op = optimizer.minimize(loss, var_list=new_cifar_list)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(flower_logits, Y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [ ]:
label.shape

And now, we are ready to train the output layer we just added, since all the other layers are frozen.   
**This may take a (very) long time to train if you are running on cpu**

Here you are going to use Tensorflow Dataset API just because creating batches for training is much easier and faster than any random minibatch function if you are writing on your own. 
For this first you need to define two placeholder for features and labels, `features_placeholder` & `labels_placeholder` respectively. You can go through tensorflow Dataset API documentation [here](https://www.tensorflow.org/programmers_guide/datasets).

In [ ]:
features_placeholder = tf.placeholder(train.dtype, shape=[None,299,299,3])
labels_placeholder = tf.placeholder(label.dtype, shape=[None,])

To avoid circumstances of mismatching number of batches and number of epochs you can use `repeat()` function from over your dataset this means it will start iterating again from first observation. After that it is required to make iterator which provides batches after every epoch using `tf.data.Dataset.make_initializable_iterator()`. Using initilizable iterator as name suggest you need to initialize it in the session before you can use it. 

In [ ]:
n_iterations_per_epoch = 100
batch_size = 30
n_epochs = int(train.shape[0]/batch_size)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
dataset = dataset.repeat().batch(batch_size)
iterator = dataset.make_initializable_iterator()
feat, lab = iterator.get_next()

In [ ]:
with tf.Session() as sess:
    init.run()
    inception_saver.restore(sess, INCEPTION_V3_CHECKPOINT_PATH)
    sess.run(iterator.initializer, feed_dict={features_placeholder: train, labels_placeholder: label})
    for epoch in range(n_epochs):
        print("Epoch", epoch, end="")
        X_batch, y_batch = sess.run([feat, lab])
        for iteration in range(n_iterations_per_epoch):
            print(".", end="")
            sess.run(training_op, feed_dict={X: X_batch, Y: y_batch, training: True})
            acc_train = accuracy.eval(feed_dict={X: X_batch, Y: y_batch})
        print(" Train accuracy:", acc_train)
    test_accuracy = accuracy.eval(feed_dict={X: train[1200].reshape((1, 299, 299,3)), Y: label[1200].reshape((1,))})
    print("Test Accuracy", test_accuracy)
    save_path = saver.save(sess, "./my_flowers_model")

# Saving and Restoring Models 
Freezing and restoring models, writing to protobuf file 

In [ ]:
saver = tf.train.import_meta_graph('./my_flowers_model.meta', clear_devices=True)
graph = tf.get_default_graph()
input_graph_def = graph.as_graph_def()
sess = tf.Session()
saver.restore(sess, "./my_flowers_model")

In [ ]:
[print(n.name) for n in tf.get_default_graph().as_graph_def().node]

In [ ]:
from tensorflow.python import graph_util

output_node_names="InceptionV3/Predictions/Softmax"

output_graph_def = graph_util.convert_variables_to_constants(sess, input_graph_def, output_node_names.split(","))

In [ ]:
output_graph="./my_flowers_model.pb"
with tf.gfile.GFile(output_graph, "wb") as f:
    f.write(output_graph_def.SerializeToString())
sess.close()

In [ ]:
frozen_graph="./dogs-cats-model.pb"
with tf.gfile.GFile(frozen_graph, "rb") as f:
    restored_graph_def = tf.GraphDef()
    restored_graph_def.ParseFromString(f.read())

In [ ]:
with tf.Graph().as_default() as graph:
    tf.import_graph_def(restored_graph_def,input_map=None,return_elements=None,name="")

In [2]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="ltdcnn")
    return graph

In [3]:
frozen_graph="./my_flowers_model.pb" 
graph = load_graph(frozen_graph)
for op in graph.get_operations():
    print(op.name)

ltdcnn/X
ltdcnn/PlaceholderWithDefault/input
ltdcnn/PlaceholderWithDefault
ltdcnn/InceptionV3/Conv2d_1a_3x3/weights
ltdcnn/InceptionV3/Conv2d_1a_3x3/weights/read
ltdcnn/InceptionV3/InceptionV3/Conv2d_1a_3x3/Conv2D
ltdcnn/InceptionV3/InceptionV3/Conv2d_1a_3x3/BatchNorm/Const
ltdcnn/InceptionV3/Conv2d_1a_3x3/BatchNorm/beta
ltdcnn/InceptionV3/Conv2d_1a_3x3/BatchNorm/beta/read
ltdcnn/InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean
ltdcnn/InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean/read
ltdcnn/InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_variance
ltdcnn/InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_variance/read
ltdcnn/InceptionV3/InceptionV3/Conv2d_1a_3x3/BatchNorm/cond/Switch
ltdcnn/InceptionV3/InceptionV3/Conv2d_1a_3x3/BatchNorm/cond/switch_t
ltdcnn/InceptionV3/InceptionV3/Conv2d_1a_3x3/BatchNorm/cond/pred_id
ltdcnn/InceptionV3/InceptionV3/Conv2d_1a_3x3/BatchNorm/cond/Const
ltdcnn/InceptionV3/InceptionV3/Conv2d_1a_3x3/BatchNorm/cond/Const_1
ltdcnn/InceptionV3/InceptionV3/Conv2d_1a_3x3/Bat

ltdcnn/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/beta
ltdcnn/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/beta/read
ltdcnn/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/moving_mean
ltdcnn/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/moving_mean/read
ltdcnn/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/moving_variance
ltdcnn/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/moving_variance/read
ltdcnn/InceptionV3/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/cond/Switch
ltdcnn/InceptionV3/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/cond/switch_t
ltdcnn/InceptionV3/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/cond/pred_id
ltdcnn/InceptionV3/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/cond/Const
ltdcnn/InceptionV3/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/cond/Const_1
ltdcnn/InceptionV3/InceptionV3/Mixed_7a/Branch_1/Conv2d_1a_3x3/BatchNorm/cond/FusedBatchNorm
ltdcnn/InceptionV3/InceptionV3/M